In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### PLOTTING FUNCTIONS

In [ ]:
# Function to plot specified variable per run
def plot_all_runs(df_list, var, color=None):
    label = var
    for df in df_list:
        df[var].plot(label=label, color=color) if color else df[var].plot(label=label)
        label="_nolegend_"  # This avoids duplicates in plot legend
    plt.xlabel("Year")
    plt.ylabel(var)
    plt.legend()

def plot_difference(df_list, var1, var2):
    label = str(var1) + " - " + str(var2)
    for df in df_list:
        diff = df[var1] - df[var2]
        diff.plot(label=label)
        label="_nolegend_"  # This avoids duplicates in plot legend
    plt.xlabel("Time")
    plt.ylabel("Amount")
    plt.legend()

def plot_avg_per_type(df_list, agent_type, region, var, color=None, label=None):
    if not label:
        label = agent_type
    for df in df_list:
        df = df[(df["Type"] == agent_type) & (df["Region"] == region)]
        mean = df.groupby(["Step"])[var].mean()
        std = df.groupby(["Step"])[var].std()
        plt.plot(mean, label=label, color=color)
        plt.fill_between(sorted(list(set(df["Step"].values))), mean, mean - std, color=color, alpha=0.2)
        plt.fill_between(sorted(list(set(df["Step"].values))), mean + std, mean, color=color, alpha=0.2)
        label="_nolegend_"  # This avoids duplicates in plot legend
    plt.xlabel("Timestep")
    plt.ylabel(var)
    plt.legend()

def plot_total_per_type(df_list, agent_type, region, var, color=None, label=None):
    if not label:
        label = agent_type
    for df in df_list:
        df = df[(df["Type"] == agent_type) & (df["Region"] == region)]
        df.groupby(["Step"])[var].sum().plot(label=label)
        label="_nolegend_"  # This avoids duplicates in plot legend
    plt.xlabel("Timestep")
    plt.ylabel(var)
    plt.legend()

### READ DATA

#### Model variables

In [ ]:
# Read all model variables to list
model_vars_df_list = []
for file in os.listdir("results/"):
    if file.startswith("model"):
        df = pd.read_csv("results/" + file)
        model_vars_df_list.append(df)

# Save timesteps where floods occur
n_steps = len(model_vars_df_list[0])
flood_timesteps = np.array(model_vars_df_list[0].index[model_vars_df_list[0]["Flood"] == True])
flood_timesteps -= 1
print("FLOODS IMPLEMENTED AT TIMESTEPS", flood_timesteps)

# Show first dataframe
model_vars_df_list[0]

#### Agent variables

In [ ]:
agent_vars_df_list = []
for file in os.listdir("results/"):
    if file.startswith("agent"):
        print(file)
        df = pd.read_csv("results/" + file)
        agent_vars_df_list.append(df)

# Split firms per type
cap_firms = agent_vars_df_list[0][agent_vars_df_list[0]["Type"] == "<class 'CRAB_agents.CapitalFirm'>"].copy()
cons_firms = agent_vars_df_list[0][agent_vars_df_list[0]["Type"] == "<class 'CRAB_agents.ConsumptionGoodFirm'>"].copy()
serv_firms = agent_vars_df_list[0][agent_vars_df_list[0]["Type"] == "<class 'CRAB_agents.ServiceFirm'>"].copy()
# Split dataframe into firms created at beginning and during runtime
old_firms = list(set(agent_vars_df_list[0][agent_vars_df_list[0]["AgentID"] < 10000]["AgentID"].values))
new_firms = list(set(agent_vars_df_list[0][agent_vars_df_list[0]["AgentID"] > 10000]["AgentID"].values))

agent_vars_df_list[0]

### SANITY CHECKS

#### HOUSEHOLDS

##### HH CONSUMPTION

In [ ]:
# Plot household consumption
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.Household'>", region=r, var="Consumption", label=f"Region {r}")
plt.vlines(flood_timesteps, ymin=0, ymax=12000, color="grey", linestyle="--", linewidth=0.7)
plt.show()

##### HH Net worth

In [ ]:
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.Household'>", region=r, var="Net worth", label=f"Region {r}")
plt.vlines(flood_timesteps, ymin=0, ymax=12000, color="grey", linestyle="--", linewidth=0.7)
plt.show()

### FIRMS

##### Sectoral demand

In [ ]:
# Plot average demand per sector
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Real demand", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Real demand", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Real demand", color="tab:green")
    plt.vlines(flood_timesteps, ymin=0, ymax=50, color="grey", linestyle="--", linewidth=0.7)
    plt.title(f"Region {r}")
    plt.show()

##### Production made vs. Demand vs. Inventories

In [ ]:
for r in set(agent_vars_df_list[0]["Region"].values):
    # Plot average demand per sector
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Real demand", color="tab:blue", label="Real demand")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Demand filled", color="tab:orange", label="Demand filled")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Production made", color="tab:red", label="Production made")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Inventories", color="tab:green", label="Inventories")
    plt.ylabel("Amount")
    plt.title(f"Region {r}, Capital firms")
    plt.show()
    
    # Plot average demand per sector
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Real demand", color="tab:blue", label="Real demand")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Demand filled", color="tab:orange", label="Demand filled")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Production made", color="tab:red", label="Production made")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Inventories", color="tab:green", label="Inventories")
    plt.ylabel("Amount")
    plt.title(f"Region {r}, Consumption Good firms")
    plt.show()
    
    # Plot average demand per sector
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Real demand", color="tab:blue", label="Real demand")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Demand filled", color="tab:orange", label="Demand filled")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Production made", color="tab:red", label="Production made")
    plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Inventories", color="tab:green", label="Inventories")
    plt.ylabel("Amount")
    plt.title(f"Region {r}, Consumption Service firms")
    plt.show()

In [ ]:
# Plot average demand per sector
plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", var="Demand unfilled", color="tab:blue")
# plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", var="Demand unfilled", color="tab:orange")
# plot_total_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", var="Demand unfilled", color="tab:green")
plt.vlines(flood_timesteps, ymin=0, ymax=50, color="grey", linestyle="--", linewidth=0.7)
# plt.xlim(0, 50)
plt.show()

In [ ]:
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", var="Inventories", color="tab:blue")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", var="Inventories", color="tab:orange")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", var="Inventories", color="tab:green")
plt.vlines(flood_timesteps, ymin=-1, ymax=15, color="grey", linestyle="--", linewidth=0.7)
plt.show()

In [ ]:
# Plot inventory level as fraction of demand
for i in range(len(agent_vars_df_list)):
    df = agent_vars_df_list[i]
    df["Inv/demand"] = df["Inventories"]/df["Real demand"]
    df = df.replace({np.inf: np.nan})
    agent_vars_df_list[i] = df
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", var="Inv/demand", color="tab:blue")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", var="Inv/demand", color="tab:orange")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", var="Inv/demand", color="tab:green")
plt.vlines(flood_timesteps, ymin=-1, ymax=20, color="grey", linestyle="--", linewidth=0.7)
plt.show()

#### LABOR

##### SIZE

In [ ]:
# Plot size (n employers) for all firms (average)
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Size", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Size", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Size", color="tab:green")
    plt.hlines(y=0, xmin=0, xmax=max(agent_vars_df_list[0]["Step"]), color="grey", linestyle="--", linewidth=1)
    plt.title(f"Region {r}")
    plt.show()

In [ ]:
# Plot average labor demand per sector
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", var="Labor demand", color="tab:blue")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", var="Labor demand", color="tab:orange")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", var="Labor demand", color="tab:green")
plt.show()

#### SALES

In [ ]:
# Plot average sales per sector
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Sales", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Sales", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Sales", color="tab:green")
    plt.title(f"Region {r}")
    plt.show()

#### PRICES

In [ ]:
# Plot average prices per sector
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Price", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Price", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Price", color="tab:green")
    plt.title(f"Region {r}")
    plt.show()

##### MARKUP

In [ ]:
# Plot average sales per sector
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", var="Markup", color="tab:blue")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", var="Markup", color="tab:orange")
plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", var="Markup", color="tab:green")
plt.show()

#### NET WORTH

In [ ]:
# Plot average net worth per sector
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Net worth", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Net worth", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Net worth", color="tab:green")
    plt.hlines(y=0, xmin=0, xmax=max(agent_vars_df_list[0]["Step"]), color="grey", linestyle="--", linewidth=1)
    plt.title(f"Region {r}")
    plt.show()

#### PRODUCTIVITY and WAGES

##### Productivity : wage ratio

In [ ]:
# Plot average ratio productivity : wage per sector
for df in agent_vars_df_list:
    df["Prod/wage"] = df["Prod"]/df["Wage"]
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Prod/wage", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Prod/wage", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Prod/wage", color="tab:green")
    plt.hlines(y=1, xmin=0, xmax=max(df["Step"]), color="grey", linestyle="--", linewidth=1)
    plt.title(f"Region {r}")
    plt.show()

##### Productivity

In [ ]:
# Plot average productivity per sector
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Machine prod", color="tab:red")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Prod", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Prod", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Prod", color="tab:green")
    plt.hlines(y=1, xmin=0, xmax=max(agent_vars_df_list[0]["Step"]), color="grey", linestyle="--", linewidth=1)
    plt.title(f"Region {r}")
    plt.show()

##### Wages

In [ ]:
# Plot wages
for r in set(agent_vars_df_list[0]["Region"].values):
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.CapitalFirm'>", region=r, var="Wage", color="tab:blue")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ConsumptionGoodFirm'>", region=r, var="Wage", color="tab:orange")
    plot_avg_per_type(agent_vars_df_list, agent_type="<class 'CRAB_agents.ServiceFirm'>", region=r, var="Wage", color="tab:green")
    plt.hlines(y=1, xmin=0, xmax=max(agent_vars_df_list[0]["Step"]), color="grey", linestyle="--", linewidth=1)
    plt.title(f"Region {r}")
    plt.show()